In [8]:
import os

import lightning
import optuna
import torch
from lightning import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from terratorch.tasks.tiled_inference import TiledInferenceParameters

from implementation.training.utils import (
    TrainingTaskHyperparameterTuningConfig,
    create_model,
)
from roofsense.bag3d import BAG3DTileStore
from roofsense.inference.tile import TiledInferenceEngine
from roofsense.training.datamodule import TrainingDataModule
from roofsense.training.task import TrainingTask

In [4]:
study_name = "optimization"
optim_log_dirpath = os.path.join(r"C:\Documents\RoofSense\logs\3dgeoinfo", study_name)

study = optuna.load_study(
    study_name="optim", storage=f"sqlite:///{optim_log_dirpath}/storage.db"
)

best_params = study.best_params
# Convert parameter format.
for param in ["lab", "tgi"]:
    best_params[f"append_{param}"] = best_params.pop(param)

config = TrainingTaskHyperparameterTuningConfig(
    # Add constant settings.
    # Encoder
    encoder="tu-resnet18d",
    zero_init_last=True,
    output_stride=16,
    # Loss
    label_smoothing=0.1,
    # Optimizer
    optimizer="AdamW",
    # LR Scheduler
    lr_scheduler="CosineAnnealingLR",
    **best_params,
)

In [5]:
# Quantitative Performance Evaluation
log_dirpath = os.path.join(os.path.dirname(optim_log_dirpath), "test")

torch.set_float32_matmul_precision("high")
torch.backends.cudnn.allow_tf32 = True

lightning.pytorch.seed_everything(0, workers=True)

model = create_model(config)
model_path = os.path.join(
    optim_log_dirpath, f"version_{study.best_trial.number}", "ckpts", "best.ckpt"
)

task: TrainingTask = TrainingTask.load_from_checkpoint(
    checkpoint_path=model_path, model=model
)

datamodule = TrainingDataModule(
    root=r"C:\Documents\RoofSense\roofsense\dataset",
    append_lab=config.append_lab,
    append_tgi=config.append_tgi,
)

trainer = Trainer(
    logger=TensorBoardLogger(save_dir=log_dirpath, name="quantitative"), benchmark=True
)
# trainer.test(task, datamodule=datamodule)

Seed set to 0
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/resnet18d.ra2_in1k)
INFO:timm.models._hub:[timm/resnet18d.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Converted input conv conv1.0 pretrained weights from 3 to 10 channel(s)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
# Qualitative Performance Evaluation
map_dirpath = os.path.join(log_dirpath, "qualitative")
os.makedirs(map_dirpath, exist_ok=True)

tile_id = "9-284-556"

TiledInferenceEngine(
    ckpt_path=model_path, tile_store=BAG3DTileStore(), model=model
).run(
    tile_id=tile_id,
    dst_filepath=os.path.join(map_dirpath, f"{tile_id}.map.pixel.tif"),
    params=TiledInferenceParameters(h_crop=512, h_stride=256, w_crop=512, w_stride=256),
)

C:\Documents\RoofSense\roofsense\utilities\file.py:70: UserWarning: The specified path: 'C:/Users/Dimit/.roofsense/2024.02.28\\9-284-556.stack.tif' does not exist in the system. The value of the 'overwrite' flag will be ignored.
  warnings.warn(msg, UserWarning)
